In [29]:
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
import pandas as pd
import numpy as np
import tarfile
import json
import matplotlib.pyplot as plt
from collections import Counter

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  
import matplotlib.pyplot as plt
%matplotlib inline
import spacy

In [46]:
tf = tarfile.open('geotagged_tweets_20160812-0912.tar.gz', mode='r')
tf.extractall()
counter = 0
tweets_dict = {}
with open('geotagged_tweets_20160812-0912.jsons') as fd:
    for line in fd:
        j_content = json.loads(line)
        tweets_dict[counter] = j_content
        counter += 1
        if counter == 3000: # to keep it  small
            break

In [47]:
df = pd.DataFrame.from_dict(tweets_dict, orient='index')
df.head()
df.columns #text and place appear to be most important

df.head()

,created_at,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,...,extended_entities,favorited,retweeted,possibly_sensitive,filter_level,lang,timestamp_ms,quoted_status_id,quoted_status_id_str,quoted_status
0,Fri Aug 12 10:04:00 +0000 2016,764039724818272256,764039724818272256,@theblaze @realDonaldTrump https://t.co/TY9DlZ...,"<a href=""http://twitter.com/download/iphone"" r...",False,7.640388e+17,764038820476051456,1.077465e+07,10774652,...,"{'media': [{'id': 764039718237409281, 'id_str'...",False,False,False,low,und,1470996240225,NaN,NaN,NaN
1,Fri Aug 12 10:04:02 +0000 2016,764039733076897792,764039733076897792,@BarackObama \n@FBI\n@LORETTALYNCH \nALL IN CO...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,None,8.132860e+05,813286,...,NaN,False,False,False,low,en,1470996242194,NaN,NaN,NaN
2,Fri Aug 12 10:04:10 +0000 2016,764039769244348417,764039769244348417,@theblaze @realDonaldTrump https://t.co/n050DB...,"<a href=""http://twitter.com/download/iphone"" r...",False,7.640388e+17,764038820476051456,1.077465e+07,10774652,...,"{'media': [{'id': 764039762973896704, 'id_str'...",False,False,False,low,und,1470996250817,NaN,NaN,NaN
3,Fri Aug 12 10:04:21 +0000 2016,764039812479225856,764039812479225856,@HillaryClinton he will do in one year all the...,"<a href=""http://twitter.com/download/iphone"" r...",False,7.626233e+17,762623255005966336,1.339836e+09,1339835893,...,NaN,False,False,NaN,low,en,1470996261125,NaN,NaN,NaN
4,Fri Aug 12 10:04:30 +0000 2016,764039849850482689,764039849850482689,#CNN #newday clear #Trump deliberately throwin...,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",False,NaN,None,NaN,None,...,NaN,False,False,NaN,low,en,1470996270035,NaN,NaN,NaN


In [48]:
df['text'][6]

'#Trump2016 #TrumpPence16 #MakeAmericaGreatAgain  https://t.co/l5UsYANVc9'

In [49]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

In [50]:
import re
from bs4 import BeautifulSoup
from nltk.tokenize import WordPunctTokenizer
tok = WordPunctTokenizer()

pat1 = r'@[A-Za-z0-9_]+'
pat2 = r'https?://[^ ]+'
combined_pat = r'|'.join((pat1, pat2))
www_pat = r'www.[^ ]+'
negations_dic = {"isn't":"is not", "aren't":"are not", "wasn't":"was not", "weren't":"were not",
                "haven't":"have not","hasn't":"has not","hadn't":"had not","won't":"will not",
                "wouldn't":"would not", "don't":"do not", "doesn't":"does not","didn't":"did not",
                "can't":"can not","couldn't":"could not","shouldn't":"should not","mightn't":"might not",
                "mustn't":"must not"}
neg_pattern = re.compile(r'\b(' + '|'.join(negations_dic.keys()) + r')\b')

def tweet_cleaner_updated(text):
    stop_words = stopwords.words('english')
    stop_words.extend(['from', 'subject', 're', 'edu', 'use','be','have','make','do','need','come','want','when','not'
                      ,'know','get','go','thing','say'])
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    try:
        bom_removed = souped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        bom_removed = souped
    stripped = re.sub(combined_pat, '', bom_removed)
    stripped = re.sub(www_pat, '', stripped)
    lower_case = stripped.lower()
    neg_handled = neg_pattern.sub(lambda x: negations_dic[x.group()], lower_case)
    letters_only = re.sub("[^a-zA-Z]", " ", neg_handled)
    # During the letters_only process two lines above, it has created unnecessay white spaces,
    # I will tokenize and join together to remove unneccessary white spaces
    words = [x for x  in tok.tokenize(letters_only) if len(x) > 1 and x not in stop_words]
    return (" ".join(words)).strip()

In [87]:
testing = df.text

test_result = []
for t in testing:
    test_result.append(tweet_cleaner_updated(t))
df['text_clean'] = test_result
test_result[:5]

['',
 'collusion together nojustice trumppence',
 '',
 'one year things done eight',
 'cnn newday clear trump deliberately throwing race knew isis destabilization mideast started iraq invasion']

In [58]:
bigram = gensim.models.Phrases(df['text_clean'], min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[df['text_clean']], threshold=100) 

bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp("".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [59]:
nlp = spacy.load('en', disable=['parser', 'ner'])

data_words_bigrams = make_bigrams(df['text_clean'])

data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])


In [61]:
data_lemmatized[:5]

[[],
 ['collusion', 'together', 'nojustice', 'trumppence'],
 [],
 ['year', 'thing', 'do'],
 ['cnn',
  'newday',
  'clear',
  'trump',
  'deliberately',
  'throw',
  'race',
  'know',
  'isis',
  'destabilization',
  'mideast',
  'start',
  'iraq',
  'invasion']]

In [62]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:10])

[[], [(0, 1), (1, 1), (2, 1), (3, 1)], [], [(4, 1), (5, 1), (6, 1)], [(7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1)], [(21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1)], [(3, 1), (20, 1), (28, 1)], [(29, 1), (30, 1), (31, 2), (32, 1), (33, 1), (34, 1), (35, 1)], [(13, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1)], [(12, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1)]]


In [83]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=4, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [84]:
# Print the Keyword in the 10 topics
print(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0, '0.037*"hillary" + 0.020*"obama" + 0.019*"isis" + 0.015*"figure" + 0.014*"tell" + 0.012*"vote" + 0.012*"keep" + 0.010*"clinton" + 0.010*"stop" + 0.009*"try"'), (1, '0.033*"nevertrump" + 0.026*"people" + 0.020*"think" + 0.013*"trump" + 0.013*"make" + 0.013*"donald" + 0.012*"man" + 0.011*"really" + 0.011*"imwithher" + 0.010*"word"'), (2, '0.063*"trump" + 0.025*"say" + 0.013*"would" + 0.011*"go" + 0.011*"president" + 0.010*"get" + 0.010*"mean" + 0.010*"time" + 0.009*"trumppence" + 0.009*"lie"'), (3, '0.033*"sarcasm" + 0.015*"tax" + 0.014*"good" + 0.013*"lose" + 0.011*"give" + 0.010*"show" + 0.009*"plan" + 0.009*"must" + 0.009*"idiot" + 0.009*"return"')]


In [85]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -8.262025281786153

Coherence Score:  0.4280139059067712


In [86]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.246310  0.074703       1        1  31.286091
0     -0.139215  0.023834       2        1  23.431332
3     -0.107222  0.140378       3        1  23.169046
1      0.000128 -0.238915       4        1  22.113533, topic_info=     Category        Freq        Term       Total  loglift  logprob
20    Default  440.000000       trump  440.000000  30.0000  30.0000
95    Default  167.000000     hillary  167.000000  29.0000  29.0000
715   Default  149.000000     sarcasm  149.000000  28.0000  28.0000
25    Default  141.000000  nevertrump  141.000000  27.0000  27.0000
215   Default  114.000000      people  114.000000  26.0000  26.0000
589   Default  155.000000         say  155.000000  25.0000  25.0000
97    Default   93.000000       obama   93.000000  24.0000  24.0000
107   Default   85.000000       think   85.000000  23.0000  23.0000
13    Default   87.000000        isis   87.000000  22.0000  22.0000
310   Default   69.000000         tax   69.000000  21.0000  21.0000
1167  Default   67.000000      figure   67.000000  20.0000  20.0000
972   Default   66.000000        tell   66.000000  19.0000  19.0000
141   Default   65.000000        good   65.000000  18.0000  18.0000
27    Default   81.000000       would   81.000000  17.0000  17.0000
255   Default   56.000000        make   56.000000  16.0000  16.0000
531   Default   57.000000        lose   57.000000  15.0000  15.0000
382   Default   54.000000      donald   54.000000  14.0000  14.0000
198   Default   56.000000        keep   56.000000  13.0000  13.0000
192   Default   53.000000         man   53.000000  12.0000  12.0000
417   Default   69.000000          go   69.000000  11.0000  11.0000
184   Default   67.000000   president   67.000000  10.0000  10.0000
521   Default   50.000000        give   50.000000   9.0000   9.0000
212   Default   49.000000      really   49.000000   8.0000   8.0000
495   Default   66.000000        vote   66.000000   7.0000   7.0000
191   Default   46.000000   imwithher   46.000000   6.0000   6.0000
78    Default   61.000000         get   61.000000   5.0000   5.0000
150   Default   47.000000     clinton   47.000000   4.0000   4.0000
381   Default   45.000000        word   45.000000   3.0000   3.0000
1082  Default   44.000000   sarcastic   44.000000   2.0000   2.0000
688   Default   59.000000        mean   59.000000   1.0000   1.0000
...       ...         ...         ...         ...      ...      ...
192    Topic4   52.433796         man   53.112160   1.4961  -4.4086
212    Topic4   48.366123      really   49.045731   1.4950  -4.4893
191    Topic4   45.936012   imwithher   46.595303   1.4947  -4.5409
381    Topic4   44.758270        word   45.437222   1.4939  -4.5668
1082   Topic4   44.069748   sarcastic   44.768063   1.4933  -4.5823
257    Topic4   37.356339      stupid   38.020515   1.4914  -4.7476
996    Topic4   38.117153        work   38.806538   1.4911  -4.7274
472    Topic4   32.910500        maga   33.566746   1.4892  -4.8743
233    Topic4   31.445379       today   32.084999   1.4888  -4.9199
234    Topic4   32.627064         way   33.307949   1.4883  -4.8830
266    Topic4   30.130337        help   30.794075   1.4872  -4.9626
276    Topic4   28.203365        sick   28.891979   1.4849  -5.0287
270    Topic4   26.076666    american   26.724112   1.4845  -5.1071
164    Topic4   25.127321  republican   25.803881   1.4824  -5.1442
87     Topic4   24.284853       crazy   24.943722   1.4822  -5.1783
56     Topic4   24.527142    election   25.211643   1.4815  -5.1683
1830   Topic4   24.025681      leader   24.708939   1.4809  -5.1890
225    Topic4   24.060097       tweet   24.748224   1.4808  -5.1876
1742   Topic4   21.278200        rain   21.910686   1.4797  -5.3104
272    Topic4   23.211456    mentally   23.906734   1.4795  -5.2235
1736   Topic4   20.418816        fine   21.052048   1.4784  -5.3517
1745   Topic4   20.053909    